In [1]:
import requests
import re
import json
import shutil
import os

import tqdm

from bs4 import BeautifulSoup

import pandas as pd

from collections import defaultdict

In [2]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.strip()

In [3]:
indexes = list(range(5000, 10000, 500)) + [53786]

data = defaultdict(list)

for idx in tqdm.tqdm_notebook(indexes):
    url = 'http://www.etoretro.ru/pic{}.htm'.format(idx)
    r = requests.get(url, allow_redirects=False)

    if r.status_code == 200: #redirecting, there is no object with corresponding id
        content = r.content.decode(r.encoding)
        
        meta_search = re.search('gMap.singlePhoto =(.*)', content, re.IGNORECASE)
        if meta_search is None: # there is no map!
            continue
        
        meta = json.loads(meta_search.group(1).strip(' ;'))
        
        if type(meta) is list: # the object is not on the map!
            continue
                
        soup = BeautifulSoup(content, 'lxml')        
    
        city = cleanhtml(re.search('Старый город:(.*)', content, re.IGNORECASE).group(1))
        title = soup.find(property='og:title').get('content')
        img_src = soup.find(property='og:image').get('content')

        img_r = requests.get(img_src, stream=True)
        img_name = img_src.split('/')[~0]
        img_path = os.path.join('images', img_name)
        with open(img_path, 'wb') as out_file:
            shutil.copyfileobj(img_r.raw, out_file)
            
        data['title'].append(title)
        data['city'].append(city)
        data['img'].append(img_name)
        data['latitude'].append(meta['latitude'])
        data['longitude'].append(meta['longitude'])
        data['year'].append(meta['year'])

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

In [5]:
meta

{'description': '',
 'id': '53786',
 'latitude': '59.948343',
 'longitude': '30.312814',
 'photo_url': 'data/thumbnails/20/1370534459c34.jpg',
 'post_url': 'pic53786.htm',
 'title': 'Петропавловская крепость (1 фото)',
 'year': '1870'}

In [107]:
pd.concat([df, df2, df3])

,city,img,latitude,longitude,title
0,Санкт-Петербург,1370534459c34.jpg,59.948343,30.312814,Петропавловская крепость
0,Санкт-Петербург,1370534459c34.jpg,59.948343,30.312814,Петропавловская крепость
0,Санкт-Петербург,1370534459c34.jpg,59.948343,30.312814,Петропавловская крепость
